# 🥋 Lekcja 13: Gradient Accumulation (Duży Batch na Małym GPU)

W PyTorch `loss.backward()` nie nadpisuje gradientów, ale je **akumuluje** (dodaje do istniejących: `w.grad += new_grad`).
Zazwyczaj walczymy z tym, wpisując `optimizer.zero_grad()` w każdej pętli.

Ale w **Gradient Accumulation** wykorzystujemy to jako zaletę!

**Algorytm:**
1.  Podziel wirtualny "Duży Batch" (np. 128) na małe "Mikro Batche" (np. 32).
2.  Zrób Forward i Backward dla Mikro Batcha.
3.  **Ważne:** Podziel Loss przez liczbę kroków akumulacji (żeby średnia się zgadzała).
4.  Powtórz N razy.
5.  Dopiero wtedy zrób `step()` i `zero_grad()`.

Dzięki temu trenujesz model tak, jakbyś miał superkomputer, używając laptopa.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

# Konfiguracja
LARGE_BATCH_SIZE = 32   # Taki chcemy symulować
MICRO_BATCH_SIZE = 8    # Taki mieści się w pamięci
ACCUMULATION_STEPS = LARGE_BATCH_SIZE // MICRO_BATCH_SIZE

print(f"Target Batch: {LARGE_BATCH_SIZE}")
print(f"Real Batch:   {MICRO_BATCH_SIZE}")
print(f"Kroki akumulacji: {ACCUMULATION_STEPS}")

# Dane i Model
data = torch.randn(LARGE_BATCH_SIZE, 10)
target = torch.randn(LARGE_BATCH_SIZE, 1)

model = nn.Linear(10, 1)
# Kopiujemy model, żeby porównać dwie metody (czy dają ten sam wynik)
model_copy = nn.Linear(10, 1)
model_copy.load_state_dict(model.state_dict())

optimizer = optim.SGD(model.parameters(), lr=0.01)
optimizer_copy = optim.SGD(model_copy.parameters(), lr=0.01)

criterion = nn.MSELoss()

Target Batch: 32
Real Batch:   8
Kroki akumulacji: 4


## Metoda 1: Standardowa (Duży Batch)

To jest nasz punkt odniesienia (Baseline).
Wrzucamy 32 próbki naraz. Zakładamy, że mamy nieskończoność RAM-u.

In [2]:
# 1. Standardowy krok (Wszystko naraz)
optimizer_copy.zero_grad()

pred_full = model_copy(data)
loss_full = criterion(pred_full, target)

loss_full.backward()
optimizer_copy.step()

print("Wagi po standardowym kroku (pierwsze 5):")
print(model_copy.weight.data[0, :5])

Wagi po standardowym kroku (pierwsze 5):
tensor([-0.2719, -0.0496, -0.2903,  0.0671,  0.2706])


## Metoda 2: Gradient Accumulation

Teraz zrobimy to samo, ale "na raty", po 8 próbek.
Kluczowe zmiany:
1.  Dzielimy `loss` przez `ACCUMULATION_STEPS`. Dlaczego?
    *   `MSELoss` liczy średnią z batcha.
    *   Średnia z 32 elementów to `sum(errors) / 32`.
    *   Średnia z 8 elementów to `sum(errors) / 8`.
    *   Jeśli po prostu dodamy gradienty z 4 małych batchy, suma będzie 4x za duża! Musimy to skorygować ręcznie.
2.  `optimizer.step()` wykonujemy tylko co N kroków.

In [3]:
# 2. Akumulacja
optimizer.zero_grad() # Zerujemy raz na początku

# Pętla po mikro-batchach
for i in range(ACCUMULATION_STEPS):
    # Wycinamy kawałek danych (Slicing)
    start = i * MICRO_BATCH_SIZE
    end = start + MICRO_BATCH_SIZE
    
    micro_data = data[start:end]
    micro_target = target[start:end]
    
    # Forward
    pred = model(micro_data)
    loss = criterion(pred, micro_target)
    
    # --- MAGIA AKUMULACJI ---
    # Normalizujemy stratę!
    loss = loss / ACCUMULATION_STEPS
    
    # Backward (Gradienty się dodają do .grad)
    loss.backward()
    
    print(f"Krok {i+1}/{ACCUMULATION_STEPS}: Gradient policzony (ale wagi stoją).")

# Dopiero teraz aktualizacja wag
optimizer.step()

print("\nWagi po akumulacji (pierwsze 5):")
print(model.weight.data[0, :5])

Krok 1/4: Gradient policzony (ale wagi stoją).
Krok 2/4: Gradient policzony (ale wagi stoją).
Krok 3/4: Gradient policzony (ale wagi stoją).
Krok 4/4: Gradient policzony (ale wagi stoją).

Wagi po akumulacji (pierwsze 5):
tensor([-0.2719, -0.0496, -0.2903,  0.0671,  0.2706])


In [4]:
# WERYFIKACJA
# Czy wyniki są identyczne?
diff = torch.abs(model.weight.data - model_copy.weight.data).max()

print("-" * 30)
print(f"Maksymalna różnica między metodami: {diff:.10f}")

if diff < 1e-6:
    print("✅ SUKCES! Akumulacja działa matematycznie identycznie jak duży batch.")
else:
    print("❌ COŚ NIE TAK. Różnica jest zbyt duża.")

------------------------------
Maksymalna różnica między metodami: 0.0000000000
✅ SUKCES! Akumulacja działa matematycznie identycznie jak duży batch.


## 🥋 Black Belt Summary

Gradient Accumulation to potężne narzędzie, ale ma **jeden haczyk**:

**Batch Normalization.**
Warstwy `BatchNorm` liczą średnią i wariancję z **bieżącego batcha**.
*   W dużym batchu (32): Statystyki są liczone z 32 próbek.
*   W akumulacji (8): Statystyki są liczone z 8 próbek (są bardziej zaszumione!).

Akumulacja symuluje duży batch dla WAG, ale **NIE dla Batchorma**.
Jeśli musisz używać akumulacji przy bardzo małych batchach (np. 1 lub 2), lepiej zamień `BatchNorm` na `LayerNorm` lub `GroupNorm`, które nie zależą od wielkości batcha.